In [2]:
!pip install textblob scikit-learn requests beautifulsoup4 transformers

  Using cached huggingface_hub-0.29.3-py3-none-any.whl.metadata (13 kB)
  Using cached tokenizers-0.21.1-cp39-abi3-win_amd64.whl.metadata (6.9 kB)
  Using cached safetensors-0.5.3-cp38-abi3-win_amd64.whl.metadata (3.9 kB)
   ---------------------------------------- 0.0/10.2 MB ? eta -:--:--
   ------------- -------------------------- 3.4/10.2 MB 16.7 MB/s eta 0:00:01
   -------------------- ------------------- 5.2/10.2 MB 13.3 MB/s eta 0:00:01
   ----------------------- ---------------- 6.0/10.2 MB 9.7 MB/s eta 0:00:01
   -------------------------- ------------- 6.8/10.2 MB 8.2 MB/s eta 0:00:01
   ----------------------------- ---------- 7.6/10.2 MB 7.1 MB/s eta 0:00:01
   -------------------------------- ------- 8.4/10.2 MB 6.5 MB/s eta 0:00:01
   ------------------------------------- -- 9.4/10.2 MB 6.2 MB/s eta 0:00:01
   ---------------------------------------  10.0/10.2 MB 6.0 MB/s eta 0:00:01
   ---------------------------------------- 10.2/10.2 MB 5.8 MB/s eta 0:00:00
Using cache

In [3]:
import re
from textblob import TextBlob
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import requests
from bs4 import BeautifulSoup
from transformers import pipeline


In [4]:
def analyze_sentiment(text):
    blob = TextBlob(text)
    sentiment = blob.sentiment.polarity  # Retourne une valeur entre -1 (négatif) et 1 (positif)
    
    if sentiment > 0:
        return 'Positif'
    elif sentiment < 0:
        return 'Négatif'
    else:
        return 'Neutre'

# Fonction pour identifier le sujet avec Latent Dirichlet Allocation (LDA)
def identify_topics(texts, n_topics=1, n_top_words=5):
    vectorizer = CountVectorizer(stop_words='english')
    dtm = vectorizer.fit_transform(texts)
    
    lda = LatentDirichletAllocation(n_components=n_topics, random_state=0)
    lda.fit(dtm)
    
    topics = []
    for idx, topic in enumerate(lda.components_):
        terms = [vectorizer.get_feature_names_out()[i] for i in topic.argsort()[:-n_top_words - 1:-1]]
        topics.extend(terms)
    return topics

def scrape_news(topics, websites):
    articles = []  
    for website in websites:
        response = requests.get(website)
        soup = BeautifulSoup(response.content, 'html.parser')

        # for link in soup.find_all('a', href=True):
        #     link_text = link.text.strip().lower()


        #     if all(topic.lower() in link_text for topic in topics):
        #         articles.append({'title': link.text.strip(), 'url': link['href']})

    return soup


In [1]:
from flask import Flask, request, jsonify

In [20]:
app = Flask(__name__)

@app.route('/fact-check', methods=['POST'], endpoint='fact_check_endpoint')
def get_tweet():
    # Get the tweet from the incoming JSON request
    data = request.json
    user_input = data.get('tweet', '')  # Retrieve the tweet (or empty string if not present)
    return user_input

if __name__ == '__main__':
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with watchdog (fsevents)
Traceback (most recent call last):
  File "/Users/naomi/anaconda3/lib/python3.11/site-packages/ipykernel_launcher.py", line 15, in <module>
    from ipykernel import kernelapp as app
  File "/Users/naomi/anaconda3/lib/python3.11/site-packages/ipykernel/__init__.py", line 5, in <module>
    from .connect import *  # noqa
    ^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/naomi/anaconda3/lib/python3.11/site-packages/ipykernel/connect.py", line 11, in <module>
    import jupyter_client
  File "/Users/naomi/anaconda3/lib/python3.11/site-packages/jupyter_client/__init__.py", line 8, in <module>
    from .asynchronous import AsyncKernelClient  # noqa
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/naomi/anaconda3/lib/python3.11/site-packages/jupyter_client/asynchronous/__init__.py", line 1, in <module>
    from .client import AsyncKernelClient  # noqa
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

SystemExit: 1

/Users/naomi/anaconda3/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3513: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [13]:
prompt =""" 
        In the context of fact checking, the user has provided the information enclosed in the tag <UserInput>.
        Justify it's validity while basing your answer on the following context enclosed in the tag <Context>.
        Provide a percentage of the confidence you have in the information provided according to the context.
        <UserInput>
        {UserInput}
        </UserInput>
        <Context>
        {Context}
        </Context>
        """

prompt = prompt.format(UserInput=UserInput, Context=Context)
print(prompt)

 
        In the context of fact checking, the user has provided the information enclosed in the tag <UserInput>.
        Justify it's validity while basing your answer on the following context enclosed in the tag <Context>.
        Provide a percentage of the confidence you have in the information provided according to the context.
        <UserInput>
        Earth is flat
        </UserInput>
        <Context>
        l characteristicsMean radius6371.0 km[6]Equatorial radius6378.137 km[7][8]Polar radius6356.752 km[9]Flattening1/298.257222101 (ETRS89)[10]Circumference40075.017 km equatorial[8]40007.86 km meridional[
        </Context>
        


In [14]:
model = pipeline("text-generation", model="gpt2")

ans = model(prompt, max_length=100, num_return_sequences=1)
print(ans[0]['generated_text'])

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

c:\Users\antoi\anaconda3\envs\MLP-Project\Lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\antoi\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


ValueError: Input length of input_ids is 100, but `max_length` is set to 100. This can lead to unexpected behavior. You should consider increasing `max_length` or, better yet, setting `max_new_tokens`.